# Reproducing in-text result 3

This result starts with the doctor utilisation from Figure 2A which is over 100% when patient load is 170 (IAT 3) and appointment times are 5 minutes (mean 5, SD 1, boundary 2).

The variant introduced is to reduce the doctor intervention during delivery so that:

* 50% delivery cases: no doctor
* 30% delivery cases: doctor but only one-third of typical intervention
* 20% delivery cases: doctor and full intervention

## Set up

In [1]:
# To run model
import PHC
from reproduction_helpers import process_results

# To import results and produce figures
import xlrd
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

# To speed up run time
from multiprocessing import Pool

In [2]:
# Paths to save image files to
output_folder = 'outputs'
save_path = os.path.join(output_folder, 'intext3.csv')

## Run model

In [3]:
# Parameters used in both models
base_model = {
    'OPD_iat': 3,
    'rep_file': 'arr170',
    'mean': 5,
    'sd': 1,
    'consult_boundary_1': 2,
    'consult_boundary_2': 2
}

# Model variants
variants = [
    {
        'doctor_delivery_scenario': False,
        'admin_to_staff_nurse': False,
        'rep_file': 'in3_normal.xls'
    },
    {
        'doctor_delivery_scenario': False,
        'admin_to_staff_nurse': True,
        'rep_file': 'in3_admin.xls'
    },
    {
        'doctor_delivery_scenario': True,
        'admin_to_staff_nurse': True,
        'rep_file': 'in3_admin_delivery.xls'
    },
]

In [4]:
# Combine dictionaries
dict_list = []
for var in variants:
    dict_list.append({**base_model, **var})

# Append 's_' to all items
for i, d in enumerate(dict_list):
    dict_list[i] = {f's_{k}': v for k, v in d.items()}

# View dictionaries
dict_list

[{'s_OPD_iat': 3,
  's_rep_file': 'in3_normal.xls',
  's_mean': 5,
  's_sd': 1,
  's_consult_boundary_1': 2,
  's_consult_boundary_2': 2,
  's_doctor_delivery_scenario': False,
  's_admin_to_staff_nurse': False},
 {'s_OPD_iat': 3,
  's_rep_file': 'in3_admin.xls',
  's_mean': 5,
  's_sd': 1,
  's_consult_boundary_1': 2,
  's_consult_boundary_2': 2,
  's_doctor_delivery_scenario': False,
  's_admin_to_staff_nurse': True},
 {'s_OPD_iat': 3,
  's_rep_file': 'in3_admin_delivery.xls',
  's_mean': 5,
  's_sd': 1,
  's_consult_boundary_1': 2,
  's_consult_boundary_2': 2,
  's_doctor_delivery_scenario': True,
  's_admin_to_staff_nurse': True}]

In [5]:
'''# Wrapper function to allow input of dictionary with pool
def wrapper(d):
    return PHC.main(**d)

# Create a process pool that uses all CPUs
with Pool() as pool:
    # Run PHC.main() using each of inputs from config
    pool.map(wrapper, dict_list)'''

'# Wrapper function to allow input of dictionary with pool\ndef wrapper(d):\n    return PHC.main(**d)\n\n# Create a process pool that uses all CPUs\nwith Pool() as pool:\n    # Run PHC.main() using each of inputs from config\n    pool.map(wrapper, dict_list)'

## Process results

In [6]:
data = process_results([i['s_rep_file'] for i in dict_list], xls=True)
data

,in3_normal,in3_admin,in3_admin_delivery
OPD patients,43947.100000,44138.300000,43930.100000
IPD patients,188.000000,184.200000,181.400000
ANC patients,365.600000,372.300000,368.300000
Del patients,351.300000,372.700000,372.600000
OPD Q wt,6.920531,7.049388,6.416563
Pharmacy Q wt,1.277924,1.279752,1.274500
Lab Q wt,3.106386,3.131800,3.114614
doc occ,1.140580,1.026931,1.009861
Lab patient list,253626.600000,253917.500000,253304.700000
OPD q len,6.690514,6.898998,6.359794


In [7]:
def get_utilisation(index_name, new_name):
    '''
    Creates dataframe with columns for utilisation (for a given row) before and
    after scenario change.

    Parameters:
    -----------
    index_name : string
        Name of row with utilisation data
    new_name : string
        To rename the utilisation row (e.g. doc occ -> doctor utilisation)

    Returns:
    --------
    util : pandas DataFrame
        Dataframe with three columns for utilisation before and after admin,
        and change in utilisation.
    '''
    # Get utilisation
    util = round(pd.DataFrame(data.loc[index_name]).T, 3)

    # Rename index
    util = util.rename_axis('Output')

    # Rename row
    util = util.rename({index_name: new_name})

    # Rename columns for clarity for readers
    util = util.rename({
        'in3_normal': 'Normal',
        'in3_admin': 'Less admin for doctor',
        'in3_admin_delivery': 'Less admin and delivery for doctor'}, axis=1)

    return util

In [8]:
# Get results for doctor utilisation and staff nurse utilisation
util = pd.concat([
    get_utilisation('doc occ', 'Doctor utilisation'),
    get_utilisation('staff nurse occ', 'Staff nurse utilisation')])

# Display results
display(util)

# Save results
util.to_csv(save_path, index=True)

,Normal,Less admin for doctor,Less admin and delivery for doctor
Output,,,
Doctor utilisation,1.141,1.027,1.010
Staff nurse utilisation,0.318,0.405,0.403
